In [515]:
%run import_packages/import_packages.ipynb

%run function/functions.ipynb

In [516]:
applications,jobs,test_users,users,users_history = import_csv()

In [517]:
df_job = create_df_job(jobs,applications,users_history,users)
df_job_unique = df_unique_job_id(users_history,jobs)
df_job = df_job.merge(df_job_unique,on="JobTitle")

df_job

,UserID,JobTitle,NB,jobID
0,23,assisting customers,1,1
1,23,manager,1,0
2,24848,manager,1,0
3,115398,manager,1,0
4,132309,manager,1,0
...,...,...,...,...
17880,1470222,project manager/program manager – application ...,1,6978
17881,1470222,"project, product manager/team lead/ sr. system...",1,6980
17882,1470222,senior manager of it corporate systems,1,7484
17883,1470245,consultant/temporary,3,6982


In [518]:
df_test = df_create_test(test_users,users_history,df_job_unique)
df_test

,UserID,JobTitle,NB,jobID
0,12924,customer service/ sales representative,1,50
1,12924,homemaker,2,51
2,237967,homemaker,1,51
3,12924,substitute teacher,1,49
4,140031,substitute teacher,1,49
...,...,...,...,...
1001,1466313,"vice president, database marketing",1,6929
1002,1466313,"vice president, marketing consulting",1,6926
1003,1471500,cocktail server,2,6984
1004,1471500,ics associate /overnight stock/truck unloader,1,6985


In [519]:
input_data = pd.concat([df_test,df_job])
input_data.reset_index(drop=True,inplace=True)

In [520]:
size=len(df_test)
test_jobid = input_data.iloc[:size].jobID
test_jobid = test_jobid.unique().tolist()

In [521]:
len(test_jobid)

824

In [522]:
sparse_user_item = csr_matrix((input_data['NB'].astype(float), 
                               (input_data['jobID'], 
                                input_data['UserID'])))


In [523]:
sparse_user_item

<9960x1472023 sparse matrix of type '<class 'numpy.float64'>'
	with 18891 stored elements in Compressed Sparse Row format>

In [524]:
# initialize a model
model = implicit.als.AlternatingLeastSquares(factors=50)

# train the model on a sparse matrix of user/item/confidence weights
model.fit(sparse_user_item)

In [525]:
predict_job=[]
predict_similarity=[]
for i in test_jobid:
    predict_job.append(model.similar_users(i,N=4)[0].tolist()[1:])
    predict_similarity.append(model.similar_users(i,N=4)[1].tolist()[1:])

In [554]:
result=pd.DataFrame({"jobID":test_jobid,"predict_job":predict_job,"predict_similarity":predict_similarity})

predict_job_title=[]
for predict in result.predict_job:
    predict_job_title.append(df_job_unique.iloc[predict].JobTitle.tolist())
    
result["predict_job_title"]=predict_job_title

In [555]:
result = df_test.merge(result,on="jobID")

In [556]:
result

,UserID,JobTitle,NB,jobID,predict_job,predict_similarity,predict_job_title
0,12924,customer service/ sales representative,1,50,"[51, 6679, 6676]","[0.8766484260559082, 0.8663613796234131, 0.866...","[homemaker, patrol officer/ technician, custom..."
1,12924,homemaker,2,51,"[50, 175, 2933]","[0.8766484260559082, 0.8376585841178894, 0.819...","[customer service/ sales representative, stude..."
2,237967,homemaker,1,51,"[50, 175, 2933]","[0.8766484260559082, 0.8376585841178894, 0.819...","[customer service/ sales representative, stude..."
3,12924,substitute teacher,1,49,"[5131, 5132, 3228]","[0.9679384231567383, 0.9679038524627686, 0.954...","[teacher's aid/ substitute, painter/prop manag..."
4,140031,substitute teacher,1,49,"[5131, 5132, 3228]","[0.9679384231567383, 0.9679038524627686, 0.954...","[teacher's aid/ substitute, painter/prop manag..."
...,...,...,...,...,...,...,...
1001,1466313,"vice president, database marketing",1,6929,"[6926, 6928, 537]","[0.9982858300209045, 0.9584630727767944, 0.911...","[vice president, marketing consulting, directo..."
1002,1466313,"vice president, marketing consulting",1,6926,"[6929, 6928, 537]","[0.9982858300209045, 0.970604658126831, 0.9132...","[vice president, database marketing, director,..."
1003,1471500,cocktail server,2,6984,"[6984, 3021, 3024]","[0.9999998807907104, 0.9998601078987122, 0.999...","[cocktail server, manager/bartender, waitress/..."
1004,1471500,ics associate /overnight stock/truck unloader,1,6985,"[6985, 3023, 3021]","[0.9999998807907104, 0.999860405921936, 0.9998...",[ics associate /overnight stock/truck unloader...
